In [1]:
!pip install tensorflow-gpu
!pip install wandb

     |████████████████████████████████| 380.8MB 44kB/s 
     |████████████████████████████████| 3.8MB 53.0MB/s 
     |████████████████████████████████| 450kB 22.4MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.2 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found exist

     |████████████████████████████████| 1.3MB 9.3MB/s 
     |████████████████████████████████| 92kB 12.2MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
     |████████████████████████████████| 256kB 38.2MB/s 
     |████████████████████████████████| 92kB 12.3MB/s 
     |████████████████████████████████| 460kB 51.8MB/s 
     |████████████████████████████████| 184kB 59.8MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 
  Created wheel for shortuuid: filename=shortuuid-0.5.0-cp36-none-any.whl size=5499 sha256=ff129c6fe98721cc195256ad1b1b7544004ffe262e6eb1ce186ba8ab3cc7a8e1
  Stored in directory: /root/.cache/pip/wheels/3f/eb/fd/69e5177f67b505e44acbd1aedfbe44b91768ee0c4cd5636576
  Created wheel for gql: filename=gql-0.1.0-cp36-none-any.whl size=5541 sha256=7dbc3b5398c9d12c204251c205fdeae8b0debd6c2c96956b7d8b95e834018f8a
  Stored in directory: /root/.cache/pip/wheels/a7/8d/65/a3247f500d675d80a01e4d2f0ee44fe99f1faef575bc2a1664
  Created wheel for subprocess32: filename=

In [0]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM, SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.optimizers import RMSprop
# from tensorflow.keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import wandb
from wandb.keras import WandbCallback
import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument("text", type=str)

# args = parser.parse_args()

In [5]:
run = wandb.init()
config = run.config
config.hidden_nodes = 128
config.batch_size = 256
config.file = "Microsoft held talks in the past few weeks to acquire software developer platform GitHub, Business Insider reports. One person familiar with the discussions between the companies told CNBC that they had been considering a joint marketing partnership valued around $35 million, and that those discussions had progressed to a possible investment or outright acquisition. It is unclear whether talks are still ongoing, but this person said that GitHub's price for a full acquisition was more than Microsoft currently wanted to pay. GitHub was last valued at $2 billion in its last funding round 2015, but the price tag for an acquisition could be $5 billion or more, based on a price that was floated last year. GitHub's tools have become essential to software developers, who use it to store code, keep track of updates and discuss issues. The privately held company has more than 23 million individual users in more than 1.5 million organizations. It was on track to book more than $200 million in subscription revenue, including more than $110 million from companies using its enterprise product, GitHub told CNBC last fall.Microsoft has reportedly flirted with buying GitHub in the past, including in 2016, although GitHub denied those reports. A partnership would give Microsoft another connection point to the developers it needs to court to build applications on its various platforms, including the Azure cloud. Microsoft could also use data from GitHub to improve its artificial intelligence producs. The talks come amid GitHub's struggle to replace CEO and founder Chris Wanstrath, who stepped down 10 months ago. Business Insider reported that Microsoft exec Nat Friedman -- who previously ran Xamarin, a developer tools start-up that Microsoft acquired in 2016 -- may take that CEO role. Google's senior VP of ads and commerce, Sridhar Ramaswamy, has also been in discussions for the job, says the report. Microsoft declined to comment on the report. GitHub did not immediately return a request for comment."
config.maxlen = 200
config.step = 3


In [0]:
# Only load first 100k charcters because we're not using memory efficiently
# text = io.open(config.file, encoding='utf-8').read()[:100000]
text = config.file
chars = sorted(list(set(text)))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [0]:
# build a sequence for every <config.step>-th character in the text

sentences = []
next_chars = []
for i in range(0, len(text) - config.maxlen, config.step):
    sentences.append(text[i: i + config.maxlen])
    next_chars.append(text[i + config.maxlen])

In [0]:
# build up one-hot encoded input x and output y where x is a character
# in the text y is the next character in the text

x = np.zeros((len(sentences), config.maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [24]:
model = Sequential()
model.add(GRU(config.hidden_nodes, input_shape=(config.maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="rmsprop")
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 128)               71424     
_________________________________________________________________
dense (Dense)                (None, 56)                7224      
Total params: 78,648
Trainable params: 78,648
Non-trainable params: 0
_________________________________________________________________


In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


class SampleText(keras.callbacks.Callback):
    def on_epoch_end(self, batch, logs={}):
        start_index = random.randint(0, len(text) - config.maxlen - 1)

        for diversity in [0.5, 1.2]:
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index: start_index + config.maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(200):
                x_pred = np.zeros((1, config.maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()

In [26]:
model.fit(x, y, batch_size=config.batch_size,
          epochs=100, callbacks=[SampleText(), WandbCallback()])

Train on 606 samples
Epoch 1/100
256/606 [===========>..................] - ETA: 8s - loss: 4.0206
----- diversity: 0.5
----- Generating with seed: "ld be $5 billion or more, based on a price that was floated last year. GitHub's tools have become essential to software developers, who use it to store code, keep track of updates and discuss issues. "
ld be $5 billion or more, based on a price that was floated last year. GitHub's tools have become essential to software developers, who use it to store code, keep track of updates and discuss issues. WpPrbzkgtbm'$dVNd'WoNiEtxBWehaga.$$pN',16kuWoy16xl1v$TyIs0xFkzphzAhf6E5C2SbikbW5p0ap-gN.1 u0f t re PrdiFCXVi-NFvOWR0cRPv0cnAOiedGTeAaC$GcgerCgsclx6-BSilRr-PdBGg0.MHbw'F2ebr0BRrxiGe6imtqpwNdpINzsAq-FWt

----- diversity: 1.2
----- Generating with seed: "ld be $5 billion or more, based on a price that was floated last year. GitHub's tools have become essential to software developers, who use it to store code, keep track of updates and discuss issue

KeyboardInterrupt: ignored